<a href="https://colab.research.google.com/github/SlickMik/ensf692Project/blob/main/ENSF_692_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
%autosave 2

Autosaving every 2 seconds


# ESNF 692 Project
### Group # : 4
##### Authors : Danish Shahid () | Muhammad Ibrahim Khan ()
---

This notebook describes the correlation of asset classes - Oil, Gold, and S&P500 for a user selected time range

---

Importing Libraries

In [152]:
import kagglehub
import numpy as np
import pandas as pd
import matplotlib as mp
from matplotlib import pyplot as plt

# Dataset Selection

Downloading Kaggle Datasets

In [153]:
# Download latest dataset versions
gold_dataset_path = kagglehub.dataset_download("faisaljanjua0555/daily-gold-price-historical-dataset")

oil_dataset_path = kagglehub.dataset_download("guillemservera/fuels-futures-data")

sp_dataset_path = kagglehub.dataset_download("youcanttouchthis/s-p-500-dataset")

# Dataset Creation

* Reading Dataset to Pandas<br/>
* Dropping unnecessary columns<br/>
* Setting index to Date<br/>
* Parsing Date

In [154]:
# read csv files into pandas
gold_dataset = pd.read_csv(gold_dataset_path + f'/gold.csv', index_col='Date', parse_dates=True)

oil_dataset = pd.read_csv(oil_dataset_path + f'/all_fuels_data.csv', index_col='date', parse_dates=True)

sp_dataset = pd.read_csv(sp_dataset_path + f'/spy.csv', index_col='Date', parse_dates=True)

Gold Dataset
* Dropping Columns
* Renaming Columns

Describing Gold Dataset

In [155]:
gold_dataset = gold_dataset.drop(columns='Currency').rename(columns = {c : f'Gold_{c}' for c in gold_dataset.columns})
gold_dataset.head(3)

,Gold_Open,Gold_High,Gold_Low,Gold_Close,Gold_Volume
Date,,,,,
2000-01-04,289.5,289.5,280.0,283.7,21621
2000-01-05,283.7,285.0,281.0,282.1,25448
2000-01-06,281.6,282.8,280.2,282.4,19055


Oil Dataset
* Dropping Columns
* Reorganizing Columns
* Renaming Columns
* Cleaning Columns

Describing Oil Dataset

In [156]:
oil_dataset = oil_dataset[oil_dataset['commodity'].str.lower().str.contains('crude')]
oil_dataset = oil_dataset.drop(columns=['ticker', 'commodity'])
oil_dataset.rename_axis(index={'date':'Date'}, inplace=True)
oil_dataset.columns = oil_dataset.columns.str.capitalize()

oil_dataset = (
    oil_dataset[['Open', 'High', 'Low', 'Close', 'Volume']]
    .rename(columns={c : f'Oil_{c.title()}' for c in oil_dataset.columns})
)
oil_dataset.head(3)

,Oil_Open,Oil_High,Oil_Low,Oil_Close,Oil_Volume
Date,,,,,
2000-08-23,31.950001,32.799999,31.950001,32.049999,79385
2000-08-24,31.900000,32.240002,31.400000,31.629999,72978
2000-08-25,31.700001,32.099998,31.320000,32.049999,44601


S&P 500 Dataset
* Dropping Columns
* Renaming Columns

Describing S&P 500 Dataset

In [157]:
sp_dataset = sp_dataset.drop(columns='Adj Close').rename(columns = {c : f'S&P500_{c}' for c in sp_dataset.columns})
sp_dataset.head(3)

,S&P500_Open,S&P500_High,S&P500_Low,S&P500_Close,S&P500_Volume
Date,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,1003200
1993-02-01,43.96875,44.25000,43.96875,44.25000,480500
1993-02-02,44.21875,44.37500,44.12500,44.34375,201300


Join Datasets

In [161]:
assets_df = sp_dataset.join(gold_dataset, how='outer').join(oil_dataset, how='outer')
assets_df.head(3)

,S&P500_Open,S&P500_High,S&P500_Low,S&P500_Close,S&P500_Volume,Gold_Open,Gold_High,Gold_Low,Gold_Close,Gold_Volume,Oil_Open,Oil_High,Oil_Low,Oil_Close,Oil_Volume
Date,,,,,,,,,,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,1003200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,43.96875,44.25000,43.96875,44.25000,480500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,44.21875,44.37500,44.12500,44.34375,201300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop Unnecessary Price and Volume Columns

In [163]:
assets_df = assets_df.drop(columns=[x for x in assets_df.columns.unique() if 'Close' not in x], axis=1)

Drop Duplicated Columns If Any

In [164]:
if assets_df.columns.duplicated().any():
  assets_df = assets_df.T.drop_duplicates().T

Set Hierarchical Index on Rows

In [165]:
assets_df = assets_df.set_index(
    pd.MultiIndex.from_arrays(
        [assets_df.index.year,assets_df.index.month, assets_df.index.day], names=['Year', 'Month', 'Day']
        )
    )

Describing Joined Dataset after Hierarchical Indexing on Rows

In [166]:
assets_df.head(3)

S&P500_Close  Gold_Close  Oil_Close
Year Month Day                                     
1993 1     29       43.93750         NaN        NaN
     2     1        44.25000         NaN        NaN
           2        44.34375         NaN        NaN

---

# User Entry

In [168]:
print("Enter a start and end date between the range of 2000-08-01 & 2020-12-24 or enter 'n' to exit program")
min_date = pd.to_datetime('2000-08-01')
max_date = pd.to_datetime('2020-12-24')
status = True;

while status:
    try:
        start_date = pd.to_datetime(input('Enter the start date (e.g YYYY-MM-DD): '))
        end_date = pd.to_datetime(input('Enter the end date (e.g YYYY-MM-DD): '))
        rolling_period = int(input("Enter rolling period (e.g 20): "))
        if not (min_date <= start_date <= max_date):
            raise ValueError(f"Start date must be between {min_date.date()} and {max_date.date()}")
        if not (min_date <= end_date <= max_date):
            raise ValueError(f"End date must be between {min_date.date()} and {max_date.date()}")
        if start_date > end_date:
            raise ValueError("Start date must be before or equal to end date.")
        if rolling_period > 200:
            raise ValueError("Rolling period must be lower than 200")
        break
    except Exception as e:
        print(f"Invalid input: {e}")


Enter a start and end date between the range of 2000-08-01 & 2020-12-24 or enter 'n' to exit program
Enter the start date (e.g YYYY-MM-DD): 2000-12-01
Enter the end date (e.g YYYY-MM-DD): 2015-12-01
Enter rolling period (e.g 20): 20


Slicing Joined Asset Data Frame by Index

In [172]:
range_asset_df = assets_df.loc[(start_date.year, start_date.month, start_date.day):(end_date.year, end_date.month, end_date.day)]
range_asset_df.head(3)

S&P500_Close  Gold_Close  Oil_Close
Year Month Day                                     
2000 12    1       132.21875       271.9  32.049999
           4       133.34375       273.8  31.299999
           5       137.71875       273.3  29.549999

---

# Analysis & Calculations

* 20-day SMA on price
* Correlation
* Beta - returns
* Seasonality heatmap - average returns
* Alpha? - returns

20 Day SMA Normalized Closing Price

In [ ]:
sma_cols = {'S&P500_Close' : f'S&P500_Close_{rolling_period}-day_SMA', 'Gold_Close' : f'Gold_Close_{rolling_period}-day_SMA', 'Oil_Close' :  f'Oil_Close_{rolling_period}-day_SMA'}

In [ ]:
range_asset_df[[*sma_cols.values()]] = (
    range_asset_df[[*sma_cols.keys()]].bfill().rolling(window=rolling_period).mean())

In [ ]:
sliced_df = range_asset_df[[*sma_cols.values()]]
((sliced_df - sliced_df.min()) / (sliced_df.max() - sliced_df.min())).plot(kind='line', figsize=(8, 4), title=f'Asset Class Normalized {rolling_period}-day SMA Closing Price')
plt.gca().spines[['top', 'right']].set_visible(False)

Assets Closing Price Correlation

In [ ]:
# calculating correlation
range_asset_df[[*sma_cols.values()]].corr()

In [ ]:
# plotting correlation

Asset SMA Closing Price Correlation

In [ ]:
# calculating correlation

In [ ]:
# plotting correlation

# Export & Matplotlib